In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2025-03-16 23:42:38.673848: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def outliers(data, fator=1.5):
    data_limpa = data.copy()
    
    for col in data_limpa.select_dtypes(include=[np.number]).columns:
        Q1 = data_limpa[col].quantile(0.25)
        Q3 = data_limpa[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - (fator * IQR)
        upper_bound = Q3 + (fator * IQR)
        data_limpa[col] = np.clip(data_limpa[col], lower_bound, upper_bound)
    
    return data_limpa

In [3]:
data_treino = pd.read_csv('/home/caio/github/k-3/data/train.csv')
data_teste = pd.read_csv('/home/caio/github/k-3/data/test.csv')

In [4]:
data_treino_limpo = outliers(data_treino)
data_teste_limpo = outliers(data_teste)

In [5]:
feature_treino = data_treino_limpo.drop(columns=['rainfall']) # Feature Treino
target_treino = data_treino.pop('rainfall')                   # Target Treino
feature_final_imperfeito = data_teste_limpo
feature_final = feature_final_imperfeito.fillna(feature_final_imperfeito.mean())

In [6]:
X_train, X_val, y_train, y_val = train_test_split(feature_treino, target_treino, test_size=0.2, random_state=69)

model_meta = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer (regressão)
])

2025-03-16 23:42:42.101622: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [7]:


# Compilar o modelo
model_meta.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mean_squared_error')


# Treinar o meta-modelo
model_meta.fit(X_train, y_train, epochs=300, batch_size=32, validation_data=(X_val, y_val)) #0.12

Epoch 1/300
55/55 [==============================] - 1s 8ms/step - loss: 5062.8960 - val_loss: 191.5755
Epoch 2/300
55/55 [==============================] - 0s 6ms/step - loss: 78.5904 - val_loss: 53.2410
Epoch 3/300
55/55 [==============================] - 0s 7ms/step - loss: 43.4262 - val_loss: 36.2626
Epoch 4/300
55/55 [==============================] - 0s 6ms/step - loss: 30.3343 - val_loss: 25.0460
Epoch 5/300
55/55 [==============================] - 0s 5ms/step - loss: 22.0513 - val_loss: 18.3111
Epoch 6/300
55/55 [==============================] - 0s 7ms/step - loss: 16.9132 - val_loss: 14.5502
Epoch 7/300
55/55 [==============================] - 0s 8ms/step - loss: 13.7777 - val_loss: 11.8607
Epoch 8/300
55/55 [==============================] - 0s 6ms/step - loss: 11.4813 - val_loss: 10.2058
Epoch 9/300
55/55 [==============================] - 1s 9ms/step - loss: 9.8423 - val_loss: 8.8583
Epoch 10/300
55/55 [==============================] - 0s 9ms/step - loss: 8.7213 - val_los

In [8]:
previsao = model_meta.predict(data_teste)
previsao1 = model_meta.predict(feature_treino)

69/69 [==============================] - 0s 3ms/step


In [9]:
caminho_destino = 'ensemble/Neural_v1.csv'
resultado = pd.DataFrame({
    'id': data_teste['id'],
    'rainfall': previsao.flatten() 
})

resultado.to_csv(caminho_destino, index=False)

In [10]:
caminho_meta = '/home/caio/github/k-3/meta_features/neural_final.csv'
meta_f = pd.DataFrame({
    'neural': previsao.flatten() 
})

meta_f.to_csv(caminho_meta, index=False)

In [11]:
caminho_meta = '/home/caio/github/k-3/meta_features/neural_teste.csv'
meta_f = pd.DataFrame({
    'neural_treino': previsao1.flatten() 
})

meta_f.to_csv(caminho_meta, index=False)